# Análisis de duración de peaks

En el siguiente notebook se presentan g´raficos que faiclitan el análisis de la ocurrencia de peaks.

Importación de librerías.

In [1]:
import sys
import os
from functools import reduce
from collections import Counter
import random

project_path = os.path.abspath('..')
sys.path.insert(1, project_path)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import datashader as ds
import holoviews as hv
from holoviews.operation.datashader import datashade

from src.utils import get_project_root
from src.data.make_dataset import get_minma_data

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

Definición de funciones para obtención de gráficos.

In [2]:
def peak_df(station, threshold, params, path=''):
    '''
    dataframe only with days with at lrast one peak
    '''
    df = get_minma_data(params, station, from_last=from_last)
    cols_to_drop = list(df.filter(regex='(?<!no validados)_.*')) 
    df = df.drop(cols_to_drop, axis=1)
    df = df.reset_index().rename(columns={"Registros no validados_SO2": "SO2", "Registros no validados_NO2": "NO2", "Registros no validados_NO": "NO", "Registros no validados_NOX": "NOX", "Registros no validados_O3": "O3", "Registros no validados_CO": "CO", "Registros no validados_MP10": "MP10","Registros no validados_MP25": "MP25", "index": "date"})
    
    mask_peaks = df["SO2"] >= threshold
    peaks_df = df[mask_peaks]
    
    peaks_df['date'] = pd.to_datetime(peaks_df['date']).dt.date # date_time to dates
    crit_days = peaks_df["date"].unique() # unique peak days array
    
    values = [ df["date"].dt.date == day for day in crit_days ] # get a mask for each crit day
    mask_crit_days = reduce(lambda x,y: np.logical_or(x,y), values)
    
    # filtering df with the mask of critical days
    peaks_df = df[mask_crit_days]
    
    return peaks_df, crit_days

In [3]:
def generate_var_hour_df( peaks_df, var_name ):
    df_var = peaks_df[["date", var_name]]
    df_var["Time"] = df_var["date"].dt.date
    df_var["Hour"] = df_var["date"].dt.hour
    df_var = df_var.pivot(index='Hour', columns='Time', values=var_name)
    return df_var

In [4]:
# funcion que genera un grafico por cada dia
def peak_day_plots(df, var_name, station, threshold=0):
    
    for c in range(len(df.columns)):
        # Plotting the time series of given dataframe
        plt.plot(df.index, df.iloc[:,[c]])
        # Plotting threshold
        if threshold > 0:
            plt.hlines(threshold, xmin=0, xmax=23, colors='r', linestyles='dashed')

        plt.title(f'{var_name} día {df.iloc[:,c].name}')

        # Providing x and y label to the chart
        plt.ylabel(f'Concentración {var_name} [ppb]')
        plt.xlabel('Hora')

        plt.show()

In [5]:
def all_var_plot(all_peaks_df, all_var_names, station):
    df0 = all_peaks_df[0].columns
    for c in range(len(all_peaks_df[0].columns)): # for each column (day)
        for df_idx in range(len(all_peaks_df)): # for each index_df (df_variable)
            df = all_peaks_df[df_idx]
            plt.plot(df.index, df.iloc[:,[c]], label=all_var_names[df_idx]) # plot with variable legend
            plt.legend()

        plt.title(f'Dia {df.iloc[:,c].name}')
#         plt.savefig(f'generated/imgs_peaks/{station}/all_variables/img_{c}')
        plt.show()

In [6]:
def all_var_plot_non_filtered(all_peaks_df, all_var_names, station):
    df0 = all_peaks_df[0].columns
    n = len(all_peaks_df[0].columns)
    for c in random.sample(range(n), 20): # for each column (day)
        for df_idx in range(len(all_peaks_df)): # for each index_df (df_variable)
            df = all_peaks_df[df_idx]
            plt.plot(df.index, df.iloc[:,[c]], label=all_var_names[df_idx]) # plot with variable legend
            plt.legend()

        plt.title(f'Dia {df.iloc[:,c].name}')
        plt.show()

In [7]:
def duracion(row):
    d, d1, d2, d3, d4, d5, d6 = row[0], row[1], row[2], row[3], row[4], row[5], row[6]
    if d==0: return 0
    elif d==1 and d1==0: return 1
    elif d==1 and d1==1 and d2==0: return 2
    elif d==1 and d1==1 and d2==1 and d3==0: return 3
    elif d==1 and d1==1 and d2==1 and d3==1 and d4==0: return 4
    elif d==1 and d1==1 and d2==1 and d3==1 and d4==1 and d5==0: return 5
    elif d==1 and d1==1 and d2==1 and d3==1 and d4==1 and d5==1 and d6==0: return 6
    elif d==1 and d1==1 and d2==1 and d3==1 and d4==1 and d5==1 and d6==1: return 7

In [8]:
def is_initial_peak(row):
    d, d_prev = row[0], row[1]
    if d_prev == 0 and d == 1: return 1 # fila actual es peak inicial
    else: return 0

In [9]:
def peak_hour_duration(peaks_df, station_name):
    # solo con dias peaks
    duracion_peaks_df = peaks_df[["date", "SO2"]]
    # quedarme solo con los peaks y sus consecutivos
    duracion_peaks_df['Hora'] = duracion_peaks_df["date"].dt.hour
    duracion_peaks_df['d'] = duracion_peaks_df["SO2"].apply(lambda x: 1 if x>=133 else 0)

    # proximas duraciones
    duracion_peaks_df['d1'] = duracion_peaks_df['d'].shift(-1)
    duracion_peaks_df['d2'] = duracion_peaks_df['d'].shift(-2)
    duracion_peaks_df['d3'] = duracion_peaks_df['d'].shift(-3)
    duracion_peaks_df['d4'] = duracion_peaks_df['d'].shift(-4)
    duracion_peaks_df['d5'] = duracion_peaks_df['d'].shift(-5)
    duracion_peaks_df['d6'] = duracion_peaks_df['d'].shift(-6)

    # duracion anterior
    duracion_peaks_df['d_prev'] = duracion_peaks_df['d'].shift(1)

    duracion_peaks_df["Duracion_total"] = duracion_peaks_df[["d","d1","d2","d3","d4","d5","d6"]].apply(duracion,axis=1)
    duracion_peaks_df["is_initial_peak"] = duracion_peaks_df[["d","d_prev"]].apply(is_initial_peak,axis=1)

    filtered_df = duracion_peaks_df[(duracion_peaks_df.SO2>133) & (duracion_peaks_df.is_initial_peak == 1)]

    tuples = [ ( row.Hora, row.Duracion_total ) for index, row in filtered_df.iterrows()] # (hora, duracion)
    # ahora un punto por cada par (hora, duracion), pero agregar una columna con la cantidad de veces que ocurre cada par
    dict_freq = Counter( tuples )
    
    fig = px.scatter(list(dict_freq.keys()), x=0,  y=1, size= list(dict_freq.values()),
              range_x = [-0.5,23.5],
              labels={
                  '0':"Hora inicio peak",
                  '1':"Duración peak en horas",
                  'size':"Frecuencia"
              },
              title=f"Hora vs Duración de peaks - {station_name}")
    return fig

In [10]:
def barplot_peaks( peaks_df, station , threshold):
    '''
    Grafica un histograma con la cantidad de ocurrencia de peaks a lo largo de las horas del día.
    '''
    df = generate_var_hour_df( peaks_df, 'SO2' ) # var_hour format df
    df = df[ df >= threshold ] # turn non peak values into NaN
    df[ df >= threshold ] = 1 # turn peak values into 1
    df = (df.sum(axis=1)).astype('int32').to_frame().rename(columns={0: 'Cant_peaks'}) # sum amount of peaks per hour among peak days
    
    # bar plot of frecuncies
    fig = px.bar( x=df.index, 
                 y=df["Cant_peaks"], 
                 title=f'Cantidad de Peaks por Hora - Estación {station}', 
                 labels={'x':'Hora', 'y':'Cantidad de Peaks'})
    fig.show()

## Estación Quintero



In [11]:
params = ['SO2','NO2', 'NO', 'NOX', 'O3', 'CO', 'MP10', 'MP25']#,'velviento'],#'dirviento']
params_quintero = ['SO2','NO2', 'NO', 'NOX', 'O3', 'CO', 'MP10', 'MP25']
params_maitenes = ['SO2','NO2', 'NO', 'NOX', 'O3', 'CO', 'MP10', 'MP25']
params_ventanas = ['SO2','NO2', 'NO', 'NOX', 'O3', 'MP10', 'MP25']
from_last = '5y'

In [12]:
quintero_peaks_df, quintero_crit_days = peak_df(station='quintero', threshold=133, params=params_quintero, path='generated/quintero_peaks.csv')

Histograma de frecuencia de peaks por cada hora para la estación Quintero.

In [13]:
barplot_peaks( quintero_peaks_df, 'Quintero', 133 )

Scatterplot de duración de peaks en horas vs el inicio de la hora correspondiente al peak, donde el tamaño de cada circulo corresponde a la cantidad de datos asociados a la tupla (Hora inicio, duracón) para la estación Quintero.

In [14]:
peak_hour_duration(quintero_peaks_df, 'Quintero')

## Estación Los Maitenes



In [15]:
# Obtención de los datos para graficar
maitenes_peaks_df, maitenes_crit_days = peak_df(station='maitenes', threshold=133, params=params_maitenes)

Histograma de frecuencia de peaks por cada hora para la estación Los Maitenes.

In [16]:
barplot_peaks( maitenes_peaks_df, 'Maitenes', 133 )

Scatterplot de duración de peaks en horas vs el inicio de la hora correspondiente al peak, donde el tamaño de cada circulo corresponde a la cantidad de datos asociados a la tupla (Hora inicio, duracón) para la estación Los Maitenes.

In [17]:
peak_hour_duration(maitenes_peaks_df, 'Maitenes')

## Estación Ventanas

Notar que la estación Ventanas tienes solo 2 peaks que superan la norma de 133 ppb de SO2.

In [18]:
# Obtención de los datos para graficar
ventanas_peaks_df, ventanas_crit_days = peak_df(station='ventanas', threshold=133, params=params_ventanas)

Histograma de frecuencia de peaks por cada hora para la estación Ventanas.

In [19]:
barplot_peaks( ventanas_peaks_df, 'Ventanas', 133 )

Scatterplot de duración de peaks en horas vs el inicio de la hora correspondiente al peak, donde el tamaño de cada circulo corresponde a la cantidad de datos asociados a la tupla (Hora inicio, duracón) para la estación Ventanas.

In [20]:
peak_hour_duration(ventanas_peaks_df, 'Ventanas')